# README.
This notebook synchronizes the database with the CSV file in whhdata.

In [7]:
import os
import glob
import pandas as pd

# Locate and load CSV from whhdata.

In [6]:
# Where to get the data.
whhdata_path = "/whhdata"
glob_search_path = os.path.join(whhdata_path, "*.csv")
csv_paths = sorted(glob.glob(glob_search_path))
csv_path = csv_paths[0]
print("Using {}".format(csv_paths))

# Load the data-frame.
df = pd.read_csv(csv_path)

Using ['/whhdata/results-measures-20190319-102143.csv']


NameError: name 'pd' is not defined